In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

###Project: Comcast Telecom Consumer Complaints###

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
projdata = pd.read_csv("../input/comcast-telecom-consumer-complaints/Comcast_telecom_complaints_data.csv")

In [ ]:
projdata.info()## Checking Data Type and Other Information of Data.

In [ ]:
projdata.isna().sum()## Checking null value in Data

In [ ]:
projdata.size

In [ ]:
projdata.shape

In [ ]:
##Changing Column Name as # and space available in name.

FinalProjData = projdata.rename(columns={'Ticket #':'Ticket','Customer Complaint':'Customer_Complaint','Received Via':'Received_Via','Zip code':'Zip_Code','Filing on Behalf of Someone':'Filing_on_Behalf_of_Someone'},inplace=False)

In [ ]:
FinalProjData.info()## Checking column name

In [ ]:
##Cleaning of irrelvant value from Ticket Column which was "comcas".
## Changing the data type of Ticket column.
FinalProjData = FinalProjData[FinalProjData['Ticket']!='comcas']
FinalProjData.Ticket = FinalProjData.Ticket.astype(int)
FinalProjData.Date = pd.to_datetime(FinalProjData.Date_month_year)

In [ ]:
##Creating Variable for Days, Month, Year
import calendar
FinalProjData['year']= FinalProjData['Date'].dt.year
FinalProjData['month']= FinalProjData['Date'].dt.month
FinalProjData['day']= FinalProjData['Date'].dt.day
FinalProjData['dayname']= FinalProjData['Date'].dt.day_name()
FinalProjData['MonthName']= FinalProjData['month'].apply(lambda x: calendar.month_name[x])
FinalProjData['Customer_Complaint']= FinalProjData['Customer_Complaint'].str.title()

FinalProjData.head(5)

In [ ]:
#Task1:Provide the trend chart for the number of complaints at monthly and daily granularity levels.
##Monthly Complaints

ComGrp = FinalProjData.groupby('MonthName')
dt= ComGrp.count()
NewMonth = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
dt = dt.reindex(NewMonth, axis=0)
dt.head(5)

In [ ]:
##Monthly Complaints Showing on Plot

from matplotlib import style

style.use("ggplot")
plt.figure(figsize=(8,6),edgecolor='black',linewidth=4)
sns.lineplot(x='MonthName',y='Ticket', data=dt, color = 'g' , linewidth = 2,marker='o')
plt.xlabel("Months")
plt.ylabel("Number of Complaints")
plt.title("MonthWise Complaints Detail")
plt.annotate('Max complaints in Jun', color='black',
            xy=(5, 1050), xycoords='data',
            xytext=(0.85, 0.85), textcoords='axes fraction',
            arrowprops=dict(facecolor='r', shrink=0.002),
            horizontalalignment='right', verticalalignment='top')
plt.xticks(rotation=45)
plt.show()

In [ ]:
##Week Wise Daily Complaints
ComGrp = FinalProjData.groupby('dayname')
dt1= ComGrp.count()
NewDay = ['Sunday','Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
dt1 = dt1.reindex(NewDay, axis=0)
dt1.head(5)

In [ ]:
style.use("ggplot")
plt.figure(figsize=(8,6),edgecolor='black',linewidth=4)
sns.lineplot(x='dayname',y='Ticket', data=dt1, color = 'g' , linewidth = 2,marker='o')
plt.xlabel("WeekDays")
plt.ylabel("Number of Complaints")
plt.title("WeekWise Daily Complaints")
plt.xticks(rotation=45)
plt.show()

In [ ]:
##Month Wise Daily Complaints
ComGrp = FinalProjData.groupby('day')
dt3= ComGrp.count()
dt3.head()

In [ ]:
style.use("ggplot")
plt.figure(figsize=(8,6),edgecolor='black',linewidth=4)
sns.lineplot(x='day',y='Ticket', data=dt3, color = 'g' , linewidth = 2,marker='o')
plt.xlabel("MonthDays")
plt.ylabel("Number of Complaints")
plt.title("MonthWise Daily Complaints")
plt.xticks(rotation=90)
plt.show()

In [ ]:
#Task:2 - Provide a table with the frequency of complaint types.
FreqTab = FinalProjData.groupby(["Customer_Complaint"])["Ticket"].count().reset_index(name="Count").sort_values(by='Count', ascending=False)
FreqTab.head(10)

In [ ]:
#Task:3 - Create a new categorical variable with value as Open and Closed. Open & Pending is to be categorized as Open and Closed & Solved is to be categorized as Closed.
FinalProjData['InsightStatus']=FinalProjData['Status'].astype(str).replace('Pending','Open').replace('Solved','Closed')

FinalProjData.info()

In [ ]:
FinalProjData.InsightStatus.value_counts()

In [ ]:
#Task4:- - Provide state wise status of complaints in a stacked bar chart. Use the categorized variable from Q3. Provide insights on:

StBar = FinalProjData.groupby(['State','InsightStatus'])["Ticket"].count().unstack().fillna(0)
#StBar.head(5)
style.use("ggplot")
plt.figure(figsize=(10,8))
StBar.plot(kind='bar', stacked=True)
plt.xlabel("State")
plt.ylabel("Complate Count")
plt.title("Statewise Complaints Status")
plt.xticks(rotation=90,ha='center')
plt.show()

In [ ]:
#Task:5 - Which state has the maximum complaints
highTab = FinalProjData.groupby(["State"])["Ticket"].count().reset_index(name="Count").sort_values(by='Count', ascending=False)
highTab.head(5)

##Georgia has maximum complaints.

In [ ]:
#Task:6 - Which state has the highest percentage of unresolved complaints
highTab2 = FinalProjData.groupby(["State",'InsightStatus'])["Ticket"].count().unstack().fillna(0).sort_values(by='Closed', ascending=False)
highTab2['Resoved_pct'] =(highTab2['Closed']/sum(highTab2['Closed'])*100).round(2)
highTab2['UnResoved_pct'] =(highTab2['Open']/sum(highTab2['Open'])*100).round(2)
highTab2.head(5)

In [ ]:
#Task:7 -  Provide the percentage of complaints resolved till date, which were received through theInternet and customer care calls.

highTab3 = FinalProjData.groupby(["Received_Via",'InsightStatus'])["Ticket"].count().unstack().fillna(0).sort_values(by='Closed', ascending=False)
highTab3['Resoved_pct'] =(highTab3['Closed']/sum(highTab3['Closed'])*100).round(2)
highTab3['UnResoved_pct'] =(highTab3['Open']/sum(highTab3['Open'])*100).round(2)
highTab3.head(5)

In [ ]:
##End of Project##